In [1]:
import sys
sys.path.append('../src')

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

### GDP growth contributions table

In [2]:
# Settings for GDP table
slist = ['A191RL', 'DPCERY', 'DDURRY', 'DNDGRY', 'DSERRY', 'A006RY',
         'A008RY', 'A011RY', 'A014RY', 'A822RY', 'A823RY', 'A829RY', 
         'A019RY', 'A020RY', 'A021RY']

n = {'A191RL': '\cbox{red!95!black} & \\textbf{Gross Domestic Product}',
     'DPCERY': '\cbox{yellow!80!orange} & \hspace{2mm} Consumer Spending',
     'DDURRY': '& \hspace{4mm} Durable Goods',
     'DNDGRY': '& \hspace{4mm} Non-durable Goods ',
     'DSERRY': '& \hspace{4mm} Services ',
     'A006RY': '\cbox{blue!70!black} & \hspace{2mm} Gross Investment',
     'A011RY': '& \hspace{4mm} Residential ',
     'A008RY': '& \hspace{4mm} Non-residential ',
     'A014RY': '& \hspace{4mm} Change in inventories ',
     'A019RY': '\cbox{green!60!black} & \hspace{2mm} Net Exports ',
     'A020RY': '& \hspace{4mm} Exports ',
     'A021RY': '& \hspace{4mm} Imports ',
     'A822RY': '\cbox{cyan!50!white} & \hspace{2mm} Government ',
     'A823RY': '& \hspace{4mm} Federal ',
     'A829RY': '& \hspace{4mm} State and Local '}


va = pd.read_csv(data_dir / 'gdpvafull.csv', parse_dates=['date']).set_index('date')

sl2 = ['GoodsTTU', 'Manufacturing', 'Construction', 'Retail trade', 'FIRE', 
       'Oth_Serv', 'Educational services, health care, and social assistance',
       'Professional and business services', 'Information', 'Government']

n2 = {'GoodsTTU': '\cbox{purple!70!blue} & \hspace{2mm} Goods and TTU ',
      'Manufacturing': '& \hspace{4mm} Manufacturing ',
      'Construction': '& \hspace{4mm} Construction ',
      'Retail trade': '& \hspace{4mm} Retail Trade ',
      'FIRE': '\cbox{red!90!white} & \hspace{2mm} FIRE+ ',
      'Oth_Serv': '\cbox{blue!90!white} & \hspace{2mm} Other Services ',
      'Educational services, health care, and social assistance': '& \hspace{4mm} Education \& Healthcare ',
      'Professional and business services': '& \hspace{4mm} Professional \& Business',
      'Information': '& \hspace{4mm} Information ',
      'Government': '\cbox{orange!80!white} & \hspace{2mm} Government '}

jb = pd.read_csv(data_dir / 'gdpjobs.csv', parse_dates=['date']).set_index('date') #.drop(pd.to_datetime('2019-10-01')

n3 = {'pop_contr': '\cbox{lime!90!green} & \hspace{2mm} Population ',
      'epop_contr': '\cbox{green!30!teal!90!black} & \hspace{2mm} Employment Rate ',
      'hours_contr': '\cbox{blue} & \hspace{2mm} Average Hours',
      'prod': '\cbox{cyan!60!white} & \hspace{2mm} Productivity '}

sl3 = ['pop_contr', 'epop_contr', 'hours_contr', 'prod']

gd = pd.read_csv(data_dir / 'gdi.csv', parse_dates=['date']).set_index('date')

n4 = {'A261RC': '& \\textbf{Gross Domestic Income} ',
      'A4002C': '\cbox{magenta!90!blue} & \hspace{2mm} Labor ',
      'W271RC': '\cbox{yellow!60!orange} & \hspace{2mm} Profit ',
      'A262RC': '\cbox{teal!60!white} & \hspace{2mm} Depreciation ',
      'indirect': '\cbox{violet} & \hspace{2mm} Indirect Taxes '}

sl4 = ['A261RC', 'A4002C', 'W271RC', 'A262RC', 'indirect']

In [3]:
df = nipa_df(retrieve_table('T10502')['Data'], slist).sort_index()
#nipa_series_codes(retrieve_table('T10502'))

In [4]:
# GDP Table (please consider revising!!)
f = data_dir / 'gdptable.tex'
os.remove(f)

date1 = {}
date = {}
data = {s: {} for s in list(df.keys())}

blankrow = '& & & & & & & & & & \\\\'

with open(f, 'a') as table:
    
    for s in list(df.keys()):
        data[s]['name'] = n[s]

    for i in range(1, 6):
        dt = df['A191RL'].index[-i]
        q = dt.quarter
        y = dt.year
        date1[i] = dt
        date[i] = f'{y} Q{q}'
        for s in list(df.keys()):
            if s == 'A191RL':
                data[s][i] = f'{df[s].iloc[-i]:.1f}'
            else:
                data[s][i] = f'{df[s].iloc[-i]:.2f}'

    for ma in [13, 41, 121]:
        for s in list(df.keys()):
            if s == 'A191RL':
                data[s][ma] = f'{df[s].iloc[-ma:].mean():.1f}'
            else:
                data[s][ma] = f'{df[s].iloc[-ma:].mean():.2f}'

    daterow = (f'& & {date[1]} & {date[2]} & {date[3]} & {date[4]} & {date[5]}'
                ' & & 3-year & 10-year & 30-year \\\\')
    table.write(daterow + '\n')
    print(daterow)

    for s in slist:
        r = (f'{n[s]} & {data[s][1]} & {data[s][2]} & {data[s][3]} & '
             f'{data[s][4]} & {data[s][5]} & & {data[s][13]} & '
             f' {data[s][41]} & {data[s][121]} \\\\')

        print(r)
        table.write(r + '\n')

    print(blankrow)
    table.write(blankrow + '\n')

    for s in list(va.keys()):
        data[s] = {}
        for i in range(1, 6):
            if date1[i] in va.index:
                data[s][i] = f'{va[s].loc[date1[i]]:.2f}'
            else:
                data[s][i] = '--'

    for ma in [13, 41, 121]:
        for s in list(va.keys()):
            data[s][ma] = f'{va[s].iloc[-ma:].mean():.2f}'

    for s in sl2:
        r = (f'{n2[s]} & {data[s][1]} & {data[s][2]} & {data[s][3]} & '
             f'{data[s][4]} & {data[s][5]} & & {data[s][13]} & '
             f' {data[s][41]} & {data[s][121]} \\\\')
        print(r)
        table.write(r + '\n')

    print(blankrow)
    table.write(blankrow + '\n')

    for s in list(jb.keys()):
        data[s] = {}
        for i in range(1, 6):
            if date1[i] in jb.index:
                data[s][i] = f'{jb[s].loc[date1[i]]:.2f}'
            else:
                data[s][i] = '--'

    for ma in [13, 41, 121]:
        for s in list(jb.keys()):
            data[s][ma] = f'{jb[s].iloc[-ma:].mean():.2f}'

    for s in sl3:
        r = (f'{n3[s]} & {data[s][1]} & {data[s][2]} & {data[s][3]} & '
             f'{data[s][4]} & {data[s][5]} & & {data[s][13]} & '
             f' {data[s][41]} & {data[s][121]} \\\\')
        print(r)
        table.write(r + '\n')

    print(blankrow)
    table.write(blankrow)

    for s in list(gd.keys()):
        data[s] = {}
        if s == 'A261RC':
            for i in range(1, 6):
                if date1[i] in gd.index:
                    data[s][i] = f'{gd[s].loc[date1[i]]:.1f}'
                else:
                    data[s][i] = '--'
        else:
            for i in range(1, 6):
                if date1[i] in gd.index:
                    data[s][i] = f'{gd[s].loc[date1[i]]:.2f}'
                else:
                    data[s][i] = '--'

    for ma in [13, 41, 121]:
        for s in list(gd.keys()):
            if s == 'A261RC':
                data[s][ma] = f'{gd[s].iloc[-ma:].mean():.1f}'
            else:
                data[s][ma] = f'{gd[s].iloc[-ma:].mean():.2f}'

    for s in sl4:
        r = (f'{n4[s]} & {data[s][1]} & {data[s][2]} & {data[s][3]} & '
             f'{data[s][4]} & {data[s][5]} & & {data[s][13]} & '
             f' {data[s][41]} & {data[s][121]} \\\\')
        if s == 'indirect':
            r = r.replace('\\\\', '')
        print(r)
        table.write(r + '\n')

& & 2021 Q4 & 2021 Q3 & 2021 Q2 & 2021 Q1 & 2020 Q4 & & 3-year & 10-year & 30-year \\
\cbox{red!95!black} & \textbf{Gross Domestic Product} & 7.0 & 2.3 & 6.7 & 6.3 & 4.5 & & 2.7 &  2.5 & 2.6 \\
\cbox{yellow!80!orange} & \hspace{2mm} Consumer Spending & 2.13 & 1.35 & 7.92 & 7.44 & 2.26 & & 1.92 &  1.72 & 1.88 \\
& \hspace{4mm} Durable Goods & 0.24 & -2.52 & 1.01 & 3.50 & 0.10 & & 0.66 &  0.53 & 0.49 \\
& \hspace{4mm} Non-durable Goods  & 0.13 & 0.30 & 1.98 & 2.19 & -0.17 & & 0.75 &  0.47 & 0.39 \\
& \hspace{4mm} Services  & 1.76 & 3.57 & 4.93 & 1.75 & 2.34 & & 0.51 &  0.73 & 1.00 \\
\cbox{blue!70!black} & \hspace{2mm} Gross Investment & 5.38 & 2.05 & -0.65 & -0.37 & 4.01 & & 0.97 &  0.94 & 0.76 \\
& \hspace{4mm} Non-residential  & 0.43 & 0.22 & 1.21 & 1.65 & 1.57 & & 0.36 &  0.53 & 0.58 \\
& \hspace{4mm} Residential  & 0.05 & -0.38 & -0.60 & 0.60 & 1.34 & & 0.19 &  0.20 & 0.08 \\
& \hspace{4mm} Change in inventories  & 4.90 & 2.20 & -1.26 & -2.62 & 1.10 & & 0.41 &  0.21 & 0.10 \\
\cbox{

### Business Investment Table and Text

In [5]:
s = ['A008RY', 'A009RY', 'Y033RY', 'Y034RY', 'B935RY', 
     'A680RY', 'A681RY', 'Y001RY', 'B985RY', 'Y006RY']

n = { 'A008RY': 'Total',
      'A009RY': '\hspace{-2mm}\cbox{yellow!50!orange}Structures',
      'Y033RY': '\hspace{-2mm}\cbox{cyan!60!white}Equipment',
      'Y034RY': '\hspace{4mm} Information processing',
      'B935RY': '\hspace{6mm} Computers and peripherals',
      'A680RY': '\hspace{4mm} Industrial equipment',
      'A681RY': '\hspace{4mm} Transportation equipment',
      'Y001RY': '\hspace{-2mm}\cbox{violet}Intellectual property products',
      'B985RY': '\hspace{4mm} Software',
      'Y006RY': '\hspace{4mm} Research and development'}

df = (nipa_df(retrieve_table('T10502')['Data'], s)
      [list(n.keys())])

data = df.iloc[-5:].iloc[::-1].T

cols = [f' {q.year} Q{q.quarter}' 
        if i == 0 else f'`{str(q.year)[2:]} Q{q.quarter}'
        for i, q in enumerate(data.columns)]

data.columns = cols
data['3-year'] = df.rolling(13).mean().iloc[-1].round(2)
data['10-year'] = df.rolling(41).mean().iloc[-1].round(2)
data['30-year'] = df.rolling(121).mean().iloc[-1].round(2)
data.index = data.index.map(n)
(data.round(2).applymap('{:,.2f}'.format)
 .to_csv(data_dir / 'businv.tex', sep='&', line_terminator='\\\ ', quotechar=' '))

In [6]:
sl = [('A008RY', 'main'), ('A009RY', 'main'), 
      ('Y033RY', 'end'), ('Y001RY', 'end'), ('A680RY', 'end')]

d = {}

for s, style in sl:
    value = df[s].iloc[-1]
    d[s] = cont_subt(value, style=style)

t_3y = cont_subt(data.loc[n['A008RY'], '3-year'], style='of')
ld = dtxt(df.index[-1])['qtr1']
ld2 = dtxt(df.index[-1])['qtr2']
val3y = df.A008RY.rolling(13).mean().iloc[-1]
vallt = df.A008RY.iloc[-1]

compare = compare_text(vallt, val3y, [0.1, 0.5, 2.0])

text = (f"Business fixed investment {d['A008RY']} GDP growth in {ld}, "+
        f"{compare} the average {t_3y} over the three years prior to the pandemic. "+
        f"In {ld}, investment in structures {d['A009RY']} GDP growth "+
        "(see\cbox{yellow!50!orange}), "+
        f"investment in equipment {d['Y033RY']} "+
        "(see\cbox{cyan!60!white}), and investment in "+
        f"intellectual property products {d['Y001RY']} "+
        "(see\cbox{violet}).")

write_txt(text_dir / 'businv.txt', text)
print(text)

Business fixed investment contributed 0.43 percentage point to GDP growth in 2021 Q4, in line with the average contribution of 0.36 percentage point over the three years prior to the pandemic. In 2021 Q4, investment in structures subtracted 0.25 percentage point from GDP growth (see\cbox{yellow!50!orange}), investment in equipment contributed 0.14 percentage point (see\cbox{cyan!60!white}), and investment in intellectual property products contributed 0.53 percentage point (see\cbox{violet}).


### Personal Income

In [7]:
series = {
 'A065RC': '\hspace{2mm}Personal income',
 'A033RC': '\hspace{-1mm}\cbox{green!75!black} Labor',
 'A034RC': '\hspace{4mm} Wages and salaries',
 'A038RC': '\hspace{4mm} Supplements',
 'CAPITAL': '\hspace{-1mm}\cbox{orange!40!yellow}Capital',
 'A041RC': "\hspace{4mm} Proprietors' income",
 'A048RC': '\hspace{4mm} Rental income',
 'W210RC': '\hspace{4mm} Personal income receipts on assets',
 'A064RC': '\hspace{4mm} Personal interest income',
 'B703RC': '\hspace{4mm} Personal dividend income',
 'WELFARE': '\hspace{-1mm}\cbox{blue!80!white}Welfare',
 'A577RC': 'Personal current transfer receipts',
 'W823RC': '\hspace{4mm} Social security',
 'W824RC': '\hspace{4mm} Medicare',
 'W729RC': '\hspace{4mm} Medicaid',
 'W825RC': '\hspace{4mm} Unemployment insurance',
 'W826RC': "\hspace{4mm} Veterans' benefits",
 'A061RC': '\hspace{4mm} Less welfare contributions'}

In [8]:
s = ['DPCERG']

d = nipa_df(retrieve_table('T20304')['Data'], s)['DPCERG']
deflator = d.iloc[-1] / d

s = [key for key, value in series.items() 
     if key not in ['CAPITAL', 'WELFARE']]

df = (nipa_df(retrieve_table('T20100')['Data'], s)
      .assign(CAPITAL = lambda x: x['A041RC'] + x['A048RC'] + x['W210RC'],
              WELFARE = lambda x: x['A577RC'] - x['A061RC'],
              A061RC = lambda x: - x['A061RC'])
      .drop(['W210RC', 'A577RC'], axis=1)
      .multiply(deflator, axis=0))
result = growth_contrib(df, 'A065RC')

data = result.iloc[-5:].iloc[::-1].T

cols = [f' {q.year} Q{q.quarter}' 
        if i == 0 else f'`{str(q.year)[2:]} Q{q.quarter}'
        for i, q in enumerate(data.columns)]

data.columns = cols
data['3-year'] = result.rolling(13).mean().iloc[-1].round(2)
data['10-year'] = result.rolling(41).mean().iloc[-1].round(2)
data['30-year'] = result.rolling(121).mean().iloc[-1].round(2)
data.index = data.index.map(series)
(data.reindex(list(series.values())).dropna().round(2).applymap('{:,.2f}'.format)
     .to_csv(data_dir / 'pi.tex', sep='&', line_terminator='\\\ ', quotechar=' '))

### International Trade Table

In [9]:
s = {'B020RC': 'Exports of goods and services',
     'A253RC': 'Exports of goods',
     'B638RC': '\hspace{2mm}Foods, feeds, and beverages',
     'A639RC': '\hspace{2mm}Industrial supplies \& materials',
     'LA000006': '\hspace{4mm}Petroleum and products',
     'A640RC': '\hspace{2mm}Capital goods, except automotive',
     'B641RC': '\hspace{2mm}Automotive vehicles, \& parts',
     'A642RC': '\hspace{2mm}Consumer goods, ex. food \& auto',
     'B690RC': '\hspace{4mm}Durable goods',
     'B691RC': '\hspace{4mm}Nondurable goods',
     'A646RC': 'Exports of services',
     'Y800RC': '\hspace{2mm}Transport',
     'Y801RC': '\hspace{2mm}Travel',
     'B684RC': '\hspace{2mm}Intellectual property charges',
     'Y802RC': '\hspace{2mm}Other business services',
     'B021RC': 'Imports of goods and services',
     'A255RC': 'Imports of goods',
     'B647RC': '\hspace{2mm}Foods, feeds, and beverages',
     'LA000004': '\hspace{2mm}Industrial supplies \& materials',
     'B648RC': '\hspace{4mm}Petroleum and products',
     'A650RC': '\hspace{2mm}Capital goods, except automotive',
     'B651RC': '\hspace{2mm}Automotive vehicles, \& parts',
     'A652RC': '\hspace{2mm}Consumer goods, ex. food \& auto',
     'B697RC': '\hspace{4mm}Durable goods',
     'B698RC': '\hspace{4mm}Nondurable goods',
     'B656RC': 'Imports of services',
     'Y804RC': '\hspace{2mm}Transport',
     'Y805RC': '\hspace{2mm}Travel',
     'B908RC': '\hspace{2mm}Intellectual property charges',
     'Y806RC': '\hspace{2mm}Other business services'}

In [10]:
df = nipa_df(retrieve_table('T40205')['Data'], s)
df['GDP'] = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC']
df = df.div(df['GDP'], axis=0) * 100

In [11]:
a16 = df.loc['2016'].mean()
a1213 = df.loc['2012-01-01': '2013-12-01'].mean()
a0506 = df.loc['2005-01-01': '2006-12-01'].mean()
a9899 = df.loc['1998-01-01': '1999-12-01'].mean()
a8993 = df.loc['1989-01-01': '1993-12-01'].mean()

latest = df.iloc[-1]
prev = df.iloc[-2]
yrago = df.iloc[-5]

data = pd.concat([latest, prev, yrago], axis=1)
data.columns = [f' {q.year} Q{q.quarter}' 
        if i == 0 else f'`{str(q.year)[2:]} Q{q.quarter}'
        for i, q in enumerate(data.columns)]
data['2016'] = a16
data['2012 --13'] = a1213
data['2005 --06'] = a0506
data['1998 --99'] = a9899
data['1989 --93'] = a8993

data = data.round(2).applymap('{:,.2f}'.format)
data.index = data.index.map(s)
(data.iloc[:-1].to_csv(data_dir / 'exim.tex', sep='&', 
                       line_terminator='\\\ ', quotechar=' '))

In [12]:
data

,2021 Q4,`21 Q3,`20 Q4,2016,2012 --13,2005 --06,1998 --99,1989 --93
Exports of goods and services,11.08,10.71,10.34,11.94,13.61,10.31,10.41,9.42
Exports of goods,7.81,7.55,7.08,7.74,9.35,7.30,7.52,6.84
"\hspace{2mm}Foods, feeds, and beverages",0.71,0.64,0.72,0.70,0.81,0.46,0.50,0.60
\hspace{2mm}Industrial supplies \& materials,2.82,2.74,2.23,2.07,2.95,1.92,1.55,1.65
\hspace{4mm}Petroleum and products,1.02,0.93,0.63,0.53,0.90,0.28,0.11,0.12
"\hspace{2mm}Capital goods, except automotive",2.24,2.26,2.19,2.78,3.21,2.84,3.27,2.61
"\hspace{2mm}Automotive vehicles, \& parts",0.63,0.59,0.71,0.80,0.90,0.77,0.79,0.67
"\hspace{2mm}Consumer goods, ex. food \& auto",1.07,0.99,0.91,1.03,1.12,0.91,0.86,0.74
\hspace{4mm}Durable goods,0.47,0.45,0.45,0.56,0.61,0.49,0.44,0.39
\hspace{4mm}Nondurable goods,0.60,0.54,0.45,0.48,0.50,0.41,0.42,0.35
